In [2]:
import pandas as pd
import numpy as np

import gzip
import os
import json
import re
import itertools

from IPython.display import Image, display
from IPython.core.display import HTML 
from tqdm import tqdm


In [3]:
def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')


meta_path = './amazon/meta_Software.json.gz'
reviews_path = os.path.join('amazon', 'Software.json.gz')

## Metadata 
<ul>
    <li><tt>asin</tt> - ID of the product, e.g. <a href="http://www.amazon.com/dp/0000031852">0000031852</a></li>
    <li><tt>title</tt> - name of the product</li>
    <li><tt>feature</tt> - bullet-point format features of the product</li>
    <li><tt>description</tt> - description of the product</li>
    <li><tt>price</tt> - price in US dollars (at time of crawl)</li>
    <li><tt>image</tt> - url of the product image</li>
    <li><tt>related</tt> - related products (also bought, also viewed, bought together, buy after viewing)</li>
    <li><tt>salesRank</tt> - sales rank information</li>
    <li><tt>brand</tt> - brand name</li>
    <li><tt>categories</tt> - list of categories the product belongs to</li>
    <li><tt>tech1</tt> - the first technical detail table of the product</li>
    <li><tt>tech2</tt> - the second technical detail table of the product</li>
    <li><tt>similar</tt> - similar product table</li>
  </ul>

In [4]:
%time df_meta = getDF(meta_path)
df_meta.head()

CPU times: user 1.54 s, sys: 67.1 ms, total: 1.61 s
Wall time: 1.71 s


,category,tech1,description,fit,title,also_buy,image,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,details
0,[],,[],,HOLT PHYSICS LESSON PRESENTATION CD-ROM QUICK ...,[],[],,HOLT. RINEHART AND WINSTON,[],"25,550 in Software (",[],Software,,</div>,.a-box-inner{background-color:#fff}#alohaBuyBo...,0030672120,NaN
1,[],,"[, <b>Latin rhythms that will get your kids si...",,"Sing, Watch, &amp; Learn Spanish (DVD + Guide)...",[],[https://images-na.ssl-images-amazon.com/image...,,McGraw Hill,[],"15,792 in Software (",[],Software,,</div>,,0071480935,NaN
2,[],,[<b>Connect is the only integrated learning sy...,,Connect with LearnSmart Access Card for Microb...,[],[https://images-na.ssl-images-amazon.com/image...,,McGraw-Hill Science/Engineering/Math,[],"16,900 in Software (",[],Software,,</div>,,007329506X,NaN
3,[],,[],,LearnSmart Standalone Access Card for Prescott...,[],[https://images-na.ssl-images-amazon.com/image...,,McGraw-Hill Education,[],"12,986 in Software (",[],Software,,</div>,,0073513458,NaN
4,[],,[<i>Anatomy &amp; Physiology Revealed Cat</i> ...,,Anatomy &amp; Physiology Revealed Student Acce...,"[0323394612, 0323227937, 1118527488]",[https://images-na.ssl-images-amazon.com/image...,,McGraw-Hill Education,[],"14,861 in Software (",[],Software,,</div>,$4.83,0073525758,NaN


## Reviews

<ul>
    <li><tt>reviewerID</tt> - ID of the reviewer, e.g. <a href="http://www.amazon.com/gp/cdp/member-reviews/A2SUAM1J3GNN3B">A2SUAM1J3GNN3B</a></li>
    <li><tt>asin</tt> - ID of the product, e.g. <a href="http://www.amazon.com/dp/0000013714">0000013714</a></li>
    <li><tt>reviewerName</tt> - name of the reviewer</li>
    <li><tt>vote</tt> - helpful votes of the review</li>
    <li><tt>style</tt> - a disctionary of the product metadata, e.g., "Format" is "Hardcover"</li>
    <li><tt>reviewText</tt> - text of the review</li>
    <li><tt>overall</tt> - rating of the product</li>
    <li><tt>summary</tt> - summary of the review</li>
    <li><tt>unixReviewTime</tt> - time of the review (unix time)</li>
    <li><tt>reviewTime</tt> - time of the review (raw)</li>
    <li><tt>image</tt> - images that users post after they have received the product</li>
  </ul>


In [5]:
%time df_review = getDF(reviews_path)
df_review.head()

CPU times: user 11.3 s, sys: 490 ms, total: 11.8 s
Wall time: 12.3 s


,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
0,4.0,True,"03 11, 2014",A240ORQ2LF9LUI,0077613252,{'Format:': ' Loose Leaf'},Michelle W,The materials arrived early and were in excell...,Material Great,1394496000,NaN,NaN
1,4.0,True,"02 23, 2014",A1YCCU0YRLS0FE,0077613252,{'Format:': ' Loose Leaf'},Rosalind White Ames,I am really enjoying this book with the worksh...,Health,1393113600,NaN,NaN
2,1.0,True,"02 17, 2014",A1BJHRQDYVAY2J,0077613252,{'Format:': ' Loose Leaf'},Allan R. Baker,"IF YOU ARE TAKING THIS CLASS DON""T WASTE YOUR ...",ARE YOU KIDING ME?,1392595200,7,NaN
3,3.0,True,"02 17, 2014",APRDVZ6QBIQXT,0077613252,{'Format:': ' Loose Leaf'},Lucy,This book was missing pages!!! Important pages...,missing pages!!,1392595200,3,NaN
4,5.0,False,"10 14, 2013",A2JZTTBSLS1QXV,0077775473,NaN,Albert V.,I have used LearnSmart and can officially say ...,Best study product out there!,1381708800,NaN,NaN


In [6]:
imgs = df_meta[pd.notnull(df_meta.image)].image
for i in range(1, 6):
    display(Image(url=imgs.iloc[i][0], width=200, height=200))


In [7]:
imgs[1]

['https://images-na.ssl-images-amazon.com/images/I/51BtLn2-L-L._SX38_SY50_CR,0,0,38,50_.jpg']

## Remover valores repetidos

In [213]:
df_review.drop_duplicates(subset=['reviewerID', 'asin'], inplace=True)
df_meta.drop_duplicates(subset=['asin'], inplace=True)


## Limpieza de datos
Para la columna price, para toda celda donde podramos encontrar el precio unicamente se deja este, si hay texto basura o no hay nada este se remplazara por la media de los precios encontrados.

In [214]:
def df_stay_with_cols(df, cols: set):
    cols = set(df.columns.values) - cols
    return df.drop(list(cols), axis=1)


df_meta = df_stay_with_cols(df_meta, {'asin', 'title', 'brand', 'price'})
df_review = df_stay_with_cols(df_review, {'overall', 'reviewerID', 'asin', 'reviewerName', 'unixReviewTime'})
display(df_meta.head())
display(df_review.head())

,title,brand,price,asin
0,HOLT PHYSICS LESSON PRESENTATION CD-ROM QUICK ...,HOLT. RINEHART AND WINSTON,.a-box-inner{background-color:#fff}#alohaBuyBo...,0030672120
1,"Sing, Watch, &amp; Learn Spanish (DVD + Guide)...",McGraw Hill,,0071480935
2,Connect with LearnSmart Access Card for Microb...,McGraw-Hill Science/Engineering/Math,,007329506X
3,LearnSmart Standalone Access Card for Prescott...,McGraw-Hill Education,,0073513458
4,Anatomy &amp; Physiology Revealed Student Acce...,McGraw-Hill Education,$4.83,0073525758


,overall,reviewerID,asin,reviewerName,unixReviewTime
0,4.0,A240ORQ2LF9LUI,0077613252,Michelle W,1394496000
1,4.0,A1YCCU0YRLS0FE,0077613252,Rosalind White Ames,1393113600
2,1.0,A1BJHRQDYVAY2J,0077613252,Allan R. Baker,1392595200
3,3.0,APRDVZ6QBIQXT,0077613252,Lucy,1392595200
4,5.0,A2JZTTBSLS1QXV,0077775473,Albert V.,1381708800


In [215]:
def get_price_or_empty_string(s):
    groups = re_dollar.search(s)
    if groups:
        return groups.group(0).replace(',', '')  # remove ',' since we'll apply float function
    return ''

def impute_sign(df, col):
    idx = df['price'].apply(lambda x: len(x) != 0)  # select non-empty cells
    re_dollar = re.compile('[\d]+[,\d\d\d]*[\.[\d]*]?')  # compile regex for matching price like 443,432.00
    serie = df.loc[idx, col].apply(get_price_or_empty_string)  # clean cells with weird text
    df.loc[idx, col] = serie  # now text will be replaced by ''
    mu = serie.apply(lambda x: float(x) if len(x) > 0 else 0).mean()  
    idx = df[col].apply(lambda x: len(x) == 0)  # df.price == ''  # works as well
    df.loc[idx, col] = mu  # replace empty values by mu
    df[col] = df[col].apply(np.float,)  # change dtype


impute_sign(df_meta, 'price')
df_meta.head()  # do we need a price conversion ?

,title,brand,price,asin
0,HOLT PHYSICS LESSON PRESENTATION CD-ROM QUICK ...,HOLT. RINEHART AND WINSTON,3.000000,0030672120
1,"Sing, Watch, &amp; Learn Spanish (DVD + Guide)...",McGraw Hill,44.962987,0071480935
2,Connect with LearnSmart Access Card for Microb...,McGraw-Hill Science/Engineering/Math,44.962987,007329506X
3,LearnSmart Standalone Access Card for Prescott...,McGraw-Hill Education,44.962987,0073513458
4,Anatomy &amp; Physiology Revealed Student Acce...,McGraw-Hill Education,4.830000,0073525758


Nos quedamos con aquellos productos que hayan sido valorados al menos n veces.

In [216]:
def filter_by_freq(df: pd.DataFrame, column: str, min_freq: int) -> pd.DataFrame:
    """Filters the DataFrame based on the value frequency in the specified column.

    :param df: DataFrame to be filtered.
    :param column: Column name that should be frequency filtered.
    :param min_freq: Minimal value frequency for the row to be accepted.
    :return: Filter serie.
    """
    freq = df[column].value_counts()  # Frequencies of each value in the column.
    frequent_values = freq[freq >= min_freq].index  # Select frequent values. Value is in the index.
    return df[column].isin(frequent_values)  # return series filter

def label_df(df, ):  # not used
    le_user = preprocessing.LabelEncoder().fit(df_dense['user'])
    le_item = preprocessing.LabelEncoder().fit(df_dense['item'])
    df.loc[:, 'user'] = le_user.transform(df_dense['user'])
    df.loc[:, 'item'] = le_item.transform(df_dense['item'])

In [217]:
filter_user = filter_by_freq(df_review, 'reviewerID', 5)
filter_item = filter_by_freq(df_review, 'asin', 5)
df_review_dense = df_review[filter_user & filter_item]

print('shape')
print('df original: ', df_review.shape, 'df_dense: ', df_review_dense.shape)
df_review_dense.head()

shape
df original:  (450578, 5) df_dense:  (21894, 5)


,overall,reviewerID,asin,reviewerName,unixReviewTime
21,4.0,A38NELQT98S4H8,0321719816,WB Halper,1287532800
22,4.0,A3QJU4FEN8PQSZ,0321719816,Grimmy,1287360000
23,5.0,ACJT8MUC0LRF0,0321719816,D. Fowler,1287187200
25,5.0,AYUF7YETYOLNX,0321719816,Bryan Newman,1286841600
26,5.0,A31ICLWQ9CSHRS,0321719816,Al Swanson,1286409600


In [218]:
def extract_rows_from_dense(df_src, df_sel, col):
    return df_src[df_src[col].isin(df_sel[col])]


df_meta_db = extract_rows_from_dense(df_meta, df_review_dense, 'asin')
df_review_db = extract_rows_from_dense(df_review, df_review_dense, 'reviewerID')
df_review_name = df_review_db.drop_duplicates(subset=['reviewerID']).copy()

### Observacion
Hay inconsistencias en los archivos, pues en el meta no hay info de algunos elementos valorados. Se eliminaran estas valoraciones.

In [250]:
sum(~df_review_db.asin.isin(df_meta_db.asin))

1604

In [254]:
print(df_review_db.shape)
df_review_db = extract_rows_from_dense(df_review_db, df_meta_db, 'asin')

(23491, 5)


In [255]:
df_review_db.shape

(21887, 5)

## Carga de datos csv a la base de datos

In [207]:
"""
ALTER TABLE producto MODIFY idProducto VARCHAR(20);
ALTER TABLE producto MODIFY nombre VARCHAR(400);
ALTER TABLE producto MODIFY marca VARCHAR(100);

show create table sociomembresia ;
ALTER TABLE sociomembresia DROP FOREIGN KEY sociomembresia_ibfk_1;
ALTER TABLE socio modify idSocio VARCHAR(40);
ALTER TABLE sociomembresia modify idSocio VARCHAR(40);
ALTER TABLE sociomembresia 
    ADD CONSTRAINT sociomembresia_ibfk_1 
    FOREIGN KEY (idSocio) REFERENCES socio (idSocio) ON DELETE CASCADE ON UPDATE CASCADE;


alter table socio modify idSocio VARCHAR(40);
ALTER TABLE socio MODIFY nombre VARCHAR(120);

DROP TABLE IF EXISTS valoracion;
CREATE TABLE valoracion (
    idSocio VARCHAR(40) COLLATE latin1_swedish_ci NOT NULL,
    idProducto VARCHAR(20) COLLATE latin1_swedish_ci NOT NULL,
    rating DOUBLE,
    unix_time VARCHAR(20),
    PRIMARY KEY(idSocio, idProducto)
);

ALTER TABLE valoracion 
    ADD CONSTRAINT valoracion_socio
    FOREIGN KEY (idSocio) REFERENCES socio(idSocio) 
        ON DELETE CASCADE ON UPDATE CASCADE;
    
ALTER TABLE valoracion 
    ADD CONSTRAINT valoracion_producto
    FOREIGN KEY (idProducto) REFERENCES producto(idProducto) 
        ON DELETE CASCADE ON UPDATE CASCADE;   
"""


'\nalter table producto modify idProducto VARCHAR(20);\nalter table producto modify nombre VARCHAR(400);\nalter table producto modify marca VARCHAR(100);\n\nshow create table sociomembresia ;\nALTER TABLE sociomembresia DROP FOREIGN KEY sociomembresia_ibfk_1;\nALTER TABLE socio modify idSocio VARCHAR(40);\nALTER TABLE sociomembresia modify idSocio VARCHAR(40);\nALTER TABLE sociomembresia \n    ADD CONSTRAINT sociomembresia_ibfk_1 \n    FOREIGN KEY (idSocio) REFERENCES socio (idSocio) ON DELETE CASCADE ON UPDATE CASCADE;\n\n\nalter table socio modify idSocio VARCHAR(40);\nALTER TABLE socio MODIFY nombre VARCHAR(120);\n\nDROP TABLE IF EXISTS valoracion;\nCREATE TABLE valoracion (\n    idSocio VARCHAR(40) COLLATE latin1_swedish_ci NOT NULL,\n    idProducto VARCHAR(20) COLLATE latin1_swedish_ci NOT NULL,\n    rating DOUBLE,\n    unix_time VARCHAR(20),\n    PRIMARY KEY(idSocio, idProducto)\n);\n\nALTER TABLE valoracion \n    ADD CONSTRAINT valoracion_socio\n    FOREIGN KEY (idSocio) REFEREN

In [175]:
!pip3 install flask flask-mysql

You should consider upgrading via the '/home/aingeru/anaconda3/bin/python -m pip install --upgrade pip' command.


In [225]:
from flask import Flask, render_template, request, jsonify
from flaskext.mysql import MySQL
from db_helper import DbHelper


In [226]:
app = Flask(__name__)
app.config.from_pyfile('config.py')

mysql = MySQL()
mysql.init_app(app)

conn = mysql.connect()
cursor = conn.cursor()

db = DbHelper(conn, cursor)

## Insercion en producto

In [227]:
query = f'''INSERT INTO producto(idProducto, nombre, marca, precioUnitario, idSubCat) 
        VALUES '''
n = df_meta_db.shape[0]
conn.begin()
for i, row in tqdm(df_meta_db.iterrows(), total=n):
    val = f""" ("{row['asin']}", "{row['title'].replace('"', '""')}", "{row['brand']}", {row['price']}, 17); """
    cursor.execute(query + val)

        
print(query + val)


100%|██████████| 4494/4494 [00:15<00:00, 284.62it/s]

INSERT INTO producto(idProducto, nombre, marca, precioUnitario, idSubCat) 
        VALUES  ("B01HAP3NJM", "CorelDRAW Home &amp; Student Suite X8 for PC (Old Version)", "Corel", 67.48, 17); 


In [228]:
# conn.rollback()
conn.commit()

In [229]:
df_review_db.head()

,overall,reviewerID,asin,reviewerName,unixReviewTime
21,4.0,A38NELQT98S4H8,0321719816,WB Halper,1287532800
22,4.0,A3QJU4FEN8PQSZ,0321719816,Grimmy,1287360000
23,5.0,ACJT8MUC0LRF0,0321719816,D. Fowler,1287187200
25,5.0,AYUF7YETYOLNX,0321719816,Bryan Newman,1286841600
26,5.0,A31ICLWQ9CSHRS,0321719816,Al Swanson,1286409600


In [230]:
df_review_db.reviewerID.value_counts()

A5JLAU2ARJ0BO     72
A680RUE1FDO8B     67
A3W4D8XOGLWUN5    66
A225G2TFM76GYX    66
A15S4XW3CRISZ5    64
                  ..
A1SRMG16HF5TLD     5
A2T4T4U20V7HSO     5
A2U80U8KAR1DL      5
A1FW6NVKN9B9VH     5
A10M0GF5ZFODYU     5
Name: reviewerID, Length: 3176, dtype: int64

In [231]:
df_review_db.reviewerID.apply(len).max()

20

In [232]:
df_review_db.reviewerName.map(str).apply(len).max()  # there's a float over there...

66

In [233]:
df_review_name.unixReviewTime.map(str).map(len).max()

10

## Insercion en socio

In [234]:
query = f'''INSERT INTO socio(idSocio, nombre) 
        VALUES '''
n = df_review_name.shape[0]
conn.begin()
for i, row in tqdm(df_review_name.iterrows(), total=n):
    val = f""" ("{row['reviewerID']}", "{str(row['reviewerName']).replace('"', '""')}"); """
    cursor.execute(query + val)
        
print(query + val)


100%|██████████| 3176/3176 [00:06<00:00, 512.39it/s]

INSERT INTO socio(idSocio, nombre) 
        VALUES  ("A30W19MJFRJE3J", "Tahoe Johnny"); 


In [235]:
#conn.rollback()
conn.commit()


In [258]:
query = f'''INSERT INTO valoracion(idSocio, idProducto, rating, unix_time) 
        VALUES '''
n = df_review_db.shape[0]
conn.begin()
for i, row in tqdm(df_review_db.iterrows(), total=n):
    val = f""" ("{row['reviewerID']}", "{row['asin']}", {row['overall']}, {row['unixReviewTime']} ); """
#     print(query + val)
    cursor.execute(query + val)
    
        



  0%|          | 0/21887 [00:00<?, ?it/s]

  0%|          | 1/21887 [00:00<48:12,  7.57it/s]

  0%|          | 65/21887 [00:00<33:49, 10.75it/s]

INSERT INTO valoracion(idSocio, idProducto, rating, unix_time) 
        VALUES  ("A38NELQT98S4H8", "0321719816", 4.0, 1287532800 ); 
INSERT INTO valoracion(idSocio, idProducto, rating, unix_time) 
        VALUES  ("A3QJU4FEN8PQSZ", "0321719816", 4.0, 1287360000 ); 
INSERT INTO valoracion(idSocio, idProducto, rating, unix_time) 
        VALUES  ("ACJT8MUC0LRF0", "0321719816", 5.0, 1287187200 ); 
INSERT INTO valoracion(idSocio, idProducto, rating, unix_time) 
        VALUES  ("AYUF7YETYOLNX", "0321719816", 5.0, 1286841600 ); 
INSERT INTO valoracion(idSocio, idProducto, rating, unix_time) 
        VALUES  ("A31ICLWQ9CSHRS", "0321719816", 5.0, 1286409600 ); 
INSERT INTO valoracion(idSocio, idProducto, rating, unix_time) 
        VALUES  ("A2BVNVJOFXGZUB", "0321719816", 5.0, 1285459200 ); 
INSERT INTO valoracion(idSocio, idProducto, rating, unix_time) 
        VALUES  ("A2JMJVNTBL7K7E", "0321719816", 5.0, 1302134400 ); 
INSERT INTO valoracion(idSocio, idProducto, rating, unix_time) 
       



  1%|          | 196/21887 [00:00<23:37, 15.30it/s]

  1%|          | 254/21887 [00:00<16:40, 21.61it/s]

INSERT INTO valoracion(idSocio, idProducto, rating, unix_time) 
        VALUES  ("A2AGUG8B7T7DP4", "B000021Z1O", 5.0, 1416441600 ); 
INSERT INTO valoracion(idSocio, idProducto, rating, unix_time) 
        VALUES  ("A21H4R2TIHTYXL", "B000021Z1O", 5.0, 1392681600 ); 
INSERT INTO valoracion(idSocio, idProducto, rating, unix_time) 
        VALUES  ("A3A2RMW9SQS60B", "B000023VV2", 5.0, 1082160000 ); 
INSERT INTO valoracion(idSocio, idProducto, rating, unix_time) 
        VALUES  ("A30H2335OM7RD6", "B000023VV2", 5.0, 1053043200 ); 
INSERT INTO valoracion(idSocio, idProducto, rating, unix_time) 
        VALUES  ("ATD2TKJE84XRM", "B000023VVY", 5.0, 968284800 ); 
INSERT INTO valoracion(idSocio, idProducto, rating, unix_time) 
        VALUES  ("A3A2RMW9SQS60B", "B000023VVY", 5.0, 1043452800 ); 
INSERT INTO valoracion(idSocio, idProducto, rating, unix_time) 
        VALUES  ("A3MVMPFBN1RFFV", "B000023VVH", 5.0, 1105747200 ); 
INSERT INTO valoracion(idSocio, idProducto, rating, unix_time) 
       



  2%|▏         | 348/21887 [00:00<11:48, 30.38it/s]

...

INSERT INTO valoracion(idSocio, idProducto, rating, unix_time) 
        VALUES  ("A169LLIDJLYB2Z", "B005LXONE0", 1.0, 1448236800 ); 
INSERT INTO valoracion(idSocio, idProducto, rating, unix_time) 
        VALUES  ("AJK49DENMS2BT", "B005LXIJ7M", 5.0, 1435968000 ); 
INSERT INTO valoracion(idSocio, idProducto, rating, unix_time) 
        VALUES  ("ANFRSLQ583FE4", "B005N2CS00", 3.0, 1428105600 ); 
INSERT INTO valoracion(idSocio, idProducto, rating, unix_time) 
        VALUES  ("A2HS9IOTBJ4A4E", "B005N2CSLY", 4.0, 1427155200 ); 
INSERT INTO valoracion(idSocio, idProducto, rating, unix_time) 
        VALUES  ("A19TKRB4LR9ZX0", "B005N6BK8W", 5.0, 1322006400 ); 
INSERT INTO valoracion(idSocio, idProducto, rating, unix_time) 
        VALUES  ("A12QFTYDRN3EYM", "B005NBPQXC", 5.0, 1329177600 ); 
INSERT INTO valoracion(idSocio, idProducto, rating, unix_time) 
        VALUES  ("A2YJDECO5R09Z4", "B005O24686", 4.0, 1451347200 ); 
INSERT INTO valoracion(idSocio, idProducto, rating, unix_time) 
       



 97%|█████████▋| 21154/21887 [00:33<00:00, 1938.97it/s]

 98%|█████████▊| 21360/21887 [00:33<00:00, 1953.95it/s]

INSERT INTO valoracion(idSocio, idProducto, rating, unix_time) 
        VALUES  ("A1DP7V50JZ48TQ", "B00E6ONJRM", 5.0, 1450483200 ); 
INSERT INTO valoracion(idSocio, idProducto, rating, unix_time) 
        VALUES  ("A1EM5LNJBNNI26", "B00E6ONJRM", 4.0, 1448755200 ); 
INSERT INTO valoracion(idSocio, idProducto, rating, unix_time) 
        VALUES  ("A335MHPI3ZWZUC", "B00E6OPE7K", 5.0, 1417996800 ); 
INSERT INTO valoracion(idSocio, idProducto, rating, unix_time) 
        VALUES  ("A365FPI90HN6JZ", "B00E6ONJJU", 5.0, 1500768000 ); 
INSERT INTO valoracion(idSocio, idProducto, rating, unix_time) 
        VALUES  ("A1U1G73EI5IRZF", "B00E6OPDU8", 5.0, 1440720000 ); 
INSERT INTO valoracion(idSocio, idProducto, rating, unix_time) 
        VALUES  ("ATEHKPKMW9DPU", "B00E7XBJ24", 3.0, 1423526400 ); 
INSERT INTO valoracion(idSocio, idProducto, rating, unix_time) 
        VALUES  ("ARK5SAGVLW3FY", "B00EAQR8I2", 4.0, 1475625600 ); 
INSERT INTO valoracion(idSocio, idProducto, rating, unix_time) 
       



 99%|█████████▊| 21580/21887 [00:33<00:00, 2021.52it/s]

100%|██████████| 21887/21887 [00:34<00:00, 642.64it/s] 

INSERT INTO valoracion(idSocio, idProducto, rating, unix_time) 
        VALUES  ("A2IOUUEAHV75EL", "B00O74Y65U", 5.0, 1474329600 ); 
INSERT INTO valoracion(idSocio, idProducto, rating, unix_time) 
        VALUES  ("APIGXQ3J742XI", "B00OBQ63GY", 5.0, 1493510400 ); 
INSERT INTO valoracion(idSocio, idProducto, rating, unix_time) 
        VALUES  ("A31DSB6BP7QMPI", "B00OBQ6PIK", 1.0, 1440201600 ); 
INSERT INTO valoracion(idSocio, idProducto, rating, unix_time) 
        VALUES  ("A1R5NYTW214KG4", "B00OBQ6HM4", 5.0, 1489968000 ); 
INSERT INTO valoracion(idSocio, idProducto, rating, unix_time) 
        VALUES  ("A31DSB6BP7QMPI", "B00OBQ6PK8", 1.0, 1440201600 ); 
INSERT INTO valoracion(idSocio, idProducto, rating, unix_time) 
        VALUES  ("A3C9VH6SNUCEXG", "B00OC0LBCU", 1.0, 1478908800 ); 
INSERT INTO valoracion(idSocio, idProducto, rating, unix_time) 
        VALUES  ("A2NU9H8H5MJZSL", "B00OC0LBCU", 1.0, 1464652800 ); 
INSERT INTO valoracion(idSocio, idProducto, rating, unix_time) 
      

In [259]:
# conn.rollback()
conn.commit()

0